In [1]:
import os
import dill

import numpy as np
import squigglepy as sq

from squigglepy.numbers import K, M, B
from squigglepy import bayes
from tqdm import tqdm
from pprint import pprint
from copy import deepcopy
from datetime import datetime as dt

exec(open('utils.py').read())
print('Loaded')

Loaded


In [2]:
with open('caches/variables.dill', 'rb') as f:
    VARS = dill.load(f)
print('Cache from: {}'.format(dt.fromtimestamp(os.path.getmtime('caches/variables.dill'))))
print('loaded default variables from cache!') # Default variables are defined in "(4) XRisk Model.ipynb"
# TODO: can do sensitivity analysis or VOI analysis over all the variables

Cache from: 2023-10-29 20:48:36.002006
loaded default variables from cache!


In [3]:
VARS['tai_years'] = np.array(VARS['tai_years'])
len(VARS)

62

In [4]:
years = range(VARS['CURRENT_YEAR'], VARS['MAX_YEAR'])

exec(open('modules/tai_risk.py').read())
print('Loaded TAI scenarios module')

exec(open('modules/nuclear.py').read())
print('Loaded nuclear scenarios module')

exec(open('modules/great_power_war.py').read())
print('Loaded great power war scenarios module')

exec(open('modules/bio.py').read())
print('Loaded bio scenarios module')

exec(open('modules/nano.py').read())
print('Loaded nano scenarios module')

exec(open('modules/supervolcano.py').read())
print('Loaded supervolcano module')

exec(open('modules/unknown_unknown.py').read())
print('Loaded unknown unknown scenarios module')

exec(open('modules/double_dip_catastrophe.py').read())
print('Loaded double dip catastrophe module')

exec(open('modules/tai_timelines.py').read())
print('Loaded TAI timelines module')

tai_years = bayes.bayesnet(load_cache_file='caches/tai_years', verbose=True)
print('loaded TAI variables from cache')
print('Cache from: {}'.format(dt.fromtimestamp(os.path.getmtime('caches/tai_years.sqcache'))))

Loaded TAI scenarios module
Loaded nuclear scenarios module
Loaded great power war scenarios module
Loaded bio scenarios module
Loaded nano scenarios module
Loaded supervolcano module
Loaded unknown unknown scenarios module
Loaded double dip catastrophe module
Loaded TAI timelines module
Loading from cache file (`caches/tai_years.sqcache`)...
...Loaded
Caching in-memory...
...Cached!
...Reducing
...Reduced!
...All done!
loaded TAI variables from cache
Cache from: 2023-10-29 20:45:28.031086


In [5]:
# TODO: Variation on these inputs?
human_population = 8*B # TODO: Animals? Chance present is net negative?
qaly_per_person = 40 # TODO: Improvements in health over time?
VARS['total_present_value'] = human_population * qaly_per_person
print('Total value at present: {} QALY'.format(numerize(VARS['total_present_value'])))

births_per_year = 100*M # TODO: Digital minds? Population decline?
qaly_per_birth = 60 # TODO: Improvements in health over time?
VARS['total_additional_value_per_year'] = births_per_year * qaly_per_birth

VARS['years_to_consider'] = 100  # TODO: Expand somehow to include more years?
total_additional_value = VARS['total_additional_value_per_year'] * VARS['years_to_consider']
print('Total additional value over 100 years: {} QALY'.format(numerize(total_additional_value)))

total_value = VARS['total_present_value'] + total_additional_value
print('Total value of future: {} QALY'.format(numerize(total_value)))

Total value at present: 320 billion QALY
Total additional value over 100 years: 600 billion QALY
Total value of future: 920 billion QALY


In [6]:
exec(open('modules/world_state_value.py').read())
print('Loaded world state valuation module')

Loaded world state valuation module


In [7]:
%%time

exec(open('modules/define_event.py').read())
print('Model loaded')

# TODO: Reduce amount of information in cache file (only need final year) to improve load speed
collectors = bayes.bayesnet(define_event,
                            find=lambda e: e['collectors'][VARS['MAX_YEAR'] - 1],
                            load_cache_file='caches/future_assessment_model_cache',
                            reload_cache=False,
                            raw=True,
                            verbose=True,
                            cores=1,
                            n=VARS['RUNS'])
collectors[0]

Model loaded
Loading from cache file (`caches/future_assessment_model_cache.sqcache`)...
...Loaded
Caching in-memory...
...Cached!
...Finding


KeyError: 'collectors'

In [8]:
value_of_future = [value_of_world_state(world_state=c, variables=VARS) for c in tqdm(collectors)]

print('EV of future: {} QALY'.format(numerize(np.mean(value_of_future))))
print('-')
pprint(sq.get_log_percentiles(value_of_future))

NameError: name 'collectors' is not defined

In [ ]:
%%time
alt_variables = deepcopy(VARS)
alt_variables['tai_years'] = [t + 5 for t in alt_variables['tai_years']] # Uniformly and universally delay TAI by 5 years with 100% success

# TODO: Be able to declare changes in variables for only particular years

print('Running intervention model...')
alt_define_event_lambda = lambda: define_event(alt_variables, verbosity=0)
alt_collectors = bayes.bayesnet(alt_define_event_lambda,
                                find=lambda e: e['collectors'][VARS['MAX_YEAR'] - 1],
                                raw=True,
                                verbose=True,
                                cores=5,
                                n=VARS['RUNS'])
print('Ready')

In [ ]:
print('Calculating value...')
value_of_alt_future = [value_of_world_state(world_state=c, variables=alt_variables) for c in tqdm(alt_collectors)]
print('Ready')

In [ ]:
print('-')
print('EV of default future: {} QALY'.format(numerize(np.mean(value_of_future))))
print('EV of alt future: {} QALY'.format(numerize(np.mean(value_of_alt_future))))
print('-')
print('Diff / Value of intervention: Alt future is {} QALY relative to default'.format(numerize(np.mean(value_of_alt_future) - np.mean(value_of_future))))
print('-')
print('Distribution of default future')
pprint(sq.get_log_percentiles(value_of_future))
print('-')
print('Distribution of alt future')
pprint(sq.get_log_percentiles(value_of_alt_future))


In [ ]:
print('File last ran: {}'.format(dt.now()))